##Environment Assessment
- Assessing current Databricks environment on AWS
- Please set Cluster Configuration as such:-  
  - Policy: Unrestricted
  - Access mode: Shared
- Please set the dapiToken from an Admin user Account, also set the Container and Storag Account Names.
- Please make sure storage is mounted and is accessible using cluster. 
- Also if you are using Cred Pass Through make sure Table Access Controls are enables on workspace level

In [0]:
import requests

In [0]:
databricks_host = dbutils.notebook.entry_point.getDbutils().notebook().getContext().browserHostName().get()

In [0]:
dbutils.widgets.text("access_token", "Enter dapi token here", "Access Token")
access_token = dbutils.widgets.get("access_token")
# dbutils.widgets.text("bucket_name", "Enter bucket name here", "Bucket Name")
# bucket_name = dbutils.widgets.get("bucket_name")

In [0]:
if not access_token or access_token.lower() == 'enter dapi token here':
    dbutils.notebook.exit("Access token Empty")

In [0]:
headers = {"Authorization": f"Bearer {access_token}"}

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, ArrayType
from pyspark.sql.functions import explode_outer, explode

In [0]:
storage_path = f"dbfs:/FileStore/Environment_Assessment"

###Users

In [0]:
def get_users():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/preview/scim/v2/Users", headers=headers)
    # print('Response status : ', response.status_code)
    user_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "Resources" in json_response.keys():
                user_lst  = json_response["Resources"]
            else:
                user_lst = [{"message": "an error occured while getting users"}]
        else:
            user_lst = [{"message": "user not found"}]
    else:
        user_lst = [{"message": "an error occured while getting users"}]
    return user_lst

In [0]:
try:
    users_lst = get_users()
    print("Users:")
    users_df = spark.createDataFrame(users_lst)
    display(users_df)
    users_path = f"{storage_path}/users.parquet"
    users_df.write.parquet(users_path, mode="overwrite")
except Exception as e:
    print(e)

Users:


active,displayName,emails,entitlements,groups,id,name,userName
true,Nikhil Kumar Velluri,"List(Map(type -> work, value -> nikhilkumar.velluri@foreveroceans.com, primary -> true))","List(Map(value -> allow-cluster-create), Map(value -> allow-instance-pool-create))","List(Map(type -> direct, $ref -> Groups/178028625269754, value -> 178028625269754, display -> admins))",1268559393822515,"Map(givenName -> Nikhil Kumar, familyName -> Velluri)",nikhilkumar.velluri@foreveroceans.com
true,Bill Crimmel,"List(Map(type -> work, value -> bill.crimmel@foreveroceans.com, primary -> true))",List(Map(value -> databricks-sql-access)),"List(Map(type -> direct, $ref -> Groups/178028625269754, value -> 178028625269754, display -> admins), Map(type -> direct, $ref -> Groups/821941977427182, value -> 821941977427182, display -> read_only))",1546300086725245,"Map(givenName -> Bill, familyName -> Crimmel)",bill.crimmel@foreveroceans.com
true,John Scharber,"List(Map(type -> work, value -> john.scharber@foreveroceans.com, primary -> true))",null,"List(Map(type -> direct, $ref -> Groups/178028625269754, value -> 178028625269754, display -> admins), Map(type -> direct, $ref -> Groups/873929654468077, value -> 873929654468077, display -> Software-Team))",2008517578761640,"Map(givenName -> John, familyName -> Scharber)",john.scharber@foreveroceans.com
true,Luis Rodriguez,"List(Map(type -> work, value -> luis.rodriguez@foreveroceans.com, primary -> true))",List(Map(value -> databricks-sql-access)),List(),2695612372031278,"Map(givenName -> Luis, familyName -> Rodriguez)",luis.rodriguez@foreveroceans.com
true,Tony White,"List(Map(type -> work, value -> tony.white@foreveroceans.com, primary -> true))","List(Map(value -> workspace-access), Map(value -> databricks-sql-access))","List(Map(type -> direct, $ref -> Groups/821941977427182, value -> 821941977427182, display -> read_only))",3111643107822617,"Map(givenName -> Tony, familyName -> White)",tony.white@foreveroceans.com
true,Dev Fivetran,"List(Map(type -> work, value -> dev.fivetran@foreveroceans.com, primary -> true))",null,"List(Map(type -> direct, $ref -> Groups/178028625269754, value -> 178028625269754, display -> admins))",3849699777731522,"Map(givenName -> Dev, familyName -> Fivetran)",dev.fivetran@foreveroceans.com
true,Devin Dennis,"List(Map(type -> work, value -> devin.dennis@foreveroceans.com, primary -> true))",null,"List(Map(type -> direct, $ref -> Groups/178028625269754, value -> 178028625269754, display -> admins), Map(type -> direct, $ref -> Groups/821941977427182, value -> 821941977427182, display -> read_only))",4080374272075846,"Map(givenName -> Devin, familyName -> Dennis)",devin.dennis@foreveroceans.com
true,Brittany Rogers,"List(Map(type -> work, value -> brittany.rogers@foreveroceans.com, primary -> true))",null,"List(Map(type -> direct, $ref -> Groups/821941977427182, value -> 821941977427182, display -> read_only))",5125916705598599,"Map(givenName -> Brittany, familyName -> Rogers)",brittany.rogers@foreveroceans.com
true,Jack Mead,"List(Map(type -> work, value -> jack.mead@foreveroceans.com, primary -> true))",null,"List(Map(type -> direct, $ref -> Groups/178028625269754, value -> 178028625269754, display -> admins), Map(type -> direct, $ref -> Groups/873929654468077, value -> 873929654468077, display -> Software-Team))",5635831880819407,"Map(givenName -> Jack, familyName -> Mead)",jack.mead@foreveroceans.com
true,Mayank Gupta,"List(Map(type -> work, value -> mayank.gupta@foreveroceans.com, primary -> true))",null,"List(Map(type -> direct, $ref -> Groups/178028625269754, value -> 178028625269754, display -> admins), Map(type -> direct, $ref -> Groups/873929654468077, value -> 873929654468077, display -> Software-Team))",5839785463797867,"Map(givenName -> Mayank, familyName -> Gupta)",mayank.gupta@foreveroceans.com


###Groups

In [0]:
def get_groups():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/preview/scim/v2/Groups", headers=headers)
    # print('Response status : ', response.status_code)
    group_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "Resources" in json_response.keys():
                group_lst  = json_response["Resources"]
            else:
                group_lst = [{"message": "an error occured while getting groups"}]
        else:
            group_lst = [{"message": "group not found"}]
    else:
        group_lst = [{"message": "an error occured while getting groups"}]
    return group_lst

In [0]:
try:
    groups_lst = get_groups()
    for group in groups_lst:
        for key, val in group.items():
            group[key] = str(val)
    print("Groups:")
    groups_df = spark.createDataFrame(groups_lst)
    display(groups_df)
    groups_path = f"{storage_path}/groups.parquet"
    groups_df.write.parquet(groups_path, mode="overwrite")
except Exception as e:
    print(e)

Groups:


displayName,entitlements,groups,id,members,meta
admins,"[{'value': 'workspace-access'}, {'value': 'databricks-sql-access'}, {'value': 'allow-cluster-create'}, {'value': 'allow-instance-pool-create'}]",[],178028625269754,"[{'display': 'Mario Dosreis', 'value': '6194687271168576', '$ref': 'Users/6194687271168576'}, {'display': 'Jack Mead', 'value': '5635831880819407', '$ref': 'Users/5635831880819407'}, {'display': 'Bill Crimmel', 'value': '1546300086725245', '$ref': 'Users/1546300086725245'}, {'display': 'Dev Fivetran', 'value': '3849699777731522', '$ref': 'Users/3849699777731522'}, {'display': 'Nikhil Kumar Velluri', 'value': '1268559393822515', '$ref': 'Users/1268559393822515'}, {'display': 'John Scharber', 'value': '2008517578761640', '$ref': 'Users/2008517578761640'}, {'display': 'Mayank Gupta', 'value': '5839785463797867', '$ref': 'Users/5839785463797867'}, {'display': 'Mathew Goldsborough', 'value': '8217428355536273', '$ref': 'Users/8217428355536273'}, {'display': 'Devin Dennis', 'value': '4080374272075846', '$ref': 'Users/4080374272075846'}, {'display': 'debanjoy mukherjee', 'value': '5937948982222196', '$ref': 'Users/5937948982222196'}, {'display': 'rajeev buggaveeti', 'value': '7490057872151603', '$ref': 'Users/7490057872151603'}]",{'resourceType': 'WorkspaceGroup'}
users,"[{'value': 'workspace-access'}, {'value': 'databricks-sql-access'}]",[],721298170546326,"[{'display': 'Brittany Rogers', 'value': '5125916705598599', '$ref': 'Users/5125916705598599'}, {'display': 'Mayank Gupta', 'value': '5839785463797867', '$ref': 'Users/5839785463797867'}, {'display': 'Gavin Key', 'value': '8992119731463304', '$ref': 'Users/8992119731463304'}, {'display': 'rajeev buggaveeti', 'value': '7490057872151603', '$ref': 'Users/7490057872151603'}, {'display': 'Bill Crimmel', 'value': '1546300086725245', '$ref': 'Users/1546300086725245'}, {'display': 'Tony White', 'value': '3111643107822617', '$ref': 'Users/3111643107822617'}, {'display': 'Jack Mead', 'value': '5635831880819407', '$ref': 'Users/5635831880819407'}, {'display': 'Jeff Milisen', 'value': '6590835783376789', '$ref': 'Users/6590835783376789'}, {'display': 'John Scharber', 'value': '2008517578761640', '$ref': 'Users/2008517578761640'}, {'display': 'FIVETRAN_USER', 'value': '8171798087181955', '$ref': 'ServicePrincipals/8171798087181955'}, {'display': 'Robert Baikie', 'value': '5880560177636254', '$ref': 'Users/5880560177636254'}, {'display': 'debanjoy mukherjee', 'value': '5937948982222196', '$ref': 'Users/5937948982222196'}, {'display': 'Nikhil Kumar Velluri', 'value': '1268559393822515', '$ref': 'Users/1268559393822515'}, {'display': 'Devin Dennis', 'value': '4080374272075846', '$ref': 'Users/4080374272075846'}, {'display': 'Mario Dosreis', 'value': '6194687271168576', '$ref': 'Users/6194687271168576'}, {'display': 'Mathew Goldsborough', 'value': '8217428355536273', '$ref': 'Users/8217428355536273'}, {'display': 'Luis Rodriguez', 'value': '2695612372031278', '$ref': 'Users/2695612372031278'}, {'display': 'Dev Fivetran', 'value': '3849699777731522', '$ref': 'Users/3849699777731522'}]",{'resourceType': 'WorkspaceGroup'}
read_only,[{'value': 'databricks-sql-access'}],[],821941977427182,"[{'display': 'Brittany Rogers', 'value': '5125916705598599', '$ref': 'Users/5125916705598599'}, {'display': 'Gavin Key', 'value': '8992119731463304', '$ref': 'Users/8992119731463304'}, {'display': 'Jeff Milisen', 'value': '6590835783376789', '$ref': 'Users/6590835783376789'}, {'display': 'Tony White', 'value': '3111643107822617', '$ref': 'Users/3111643107822617'}, {'display': 'Devin Dennis', 'value': '4080374272075846', '$ref': 'Users/4080374272075846'}, {'display': 'Robert Baikie', 'value': '5880560177636254', '$ref': 'Users/5880560177636254'}, {'display': 'Bill Crimmel', 'value': '1546300086725245', '$ref': 'Users/1546300086725245'}]",{'resourceType': 'Group'}
Software-Team,"[{'value': 'workspace-access'}, {'value': 'databricks-sql-access'}, {'value': 'allow-cluster-create'}]",[],873929654468077,"

###ServicePrincipals

In [0]:
def get_service_principals():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/preview/scim/v2/ServicePrincipals", headers=headers)
    # print('Response status : ', response.status_code)
    service_principal_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "Resources" in json_response.keys():
                service_principal_lst  = json_response["Resources"]
            else:
                service_principal_lst = [{"message": "an error occured while getting ServicePrincipals"}]
        else:
            service_principal_lst = [{"message": "servicePrincipal not found"}]
    else:
        service_principal_lst = [{"message": "an error occured while getting ServicePrincipals"}]
    return service_principal_lst

In [0]:
try:
    service_principals_lst = get_service_principals()
    for element in service_principals_lst:
        for key, val in element.items():
            element[key] = str(val)
    print("ServicePrincipals:")
    service_principals_df = spark.createDataFrame(service_principals_lst)
    display(service_principals_df)
    service_principals_path = f"{storage_path}/service_principals.parquet"
    service_principals_df.write.parquet(service_principals_path, mode="overwrite")
except Exception as e:
    print(e)

ServicePrincipals:


active,applicationId,displayName,entitlements,groups,id
True,fc8d01a1-d467-44d0-9678-4fb11cb7dddf,FIVETRAN_USER,"[{'value': 'workspace-access'}, {'value': 'databricks-sql-access'}]",[],8171798087181955


###Clusters

In [0]:
def get_clusters():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/clusters/list", headers=headers)
    # print('Response status : ', response.status_code)
    cluster_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "clusters" in json_response.keys():
                cluster_lst  = json_response["clusters"]
            else:
                cluster_lst = [{"message": "an error occured while getting clusters"}]
        else:
            cluster_lst = [{"message": "cluster not found"}]
    else:
        cluster_lst = [{"message": "an error occured while getting clusters"}]
    return cluster_lst

In [0]:
try:
    clusters_lst = get_clusters()
    for cluster in clusters_lst:
        for key, val in cluster.items():
            cluster[key] = str(val)
    print("Clusters:")
    clusters_df = spark.createDataFrame(clusters_lst)
    display(clusters_df)
    clusters_path = f"{storage_path}/clusters.parquet"
    clusters_df.write.parquet(clusters_path, mode="overwrite")
except Exception as e:
    print(e)

Clusters:


autotermination_minutes,aws_attributes,cluster_cores,cluster_id,cluster_memory_mb,cluster_name,cluster_source,creator_user_name,data_security_mode,default_tags,disk_spec,driver,driver_healthy,driver_instance_source,driver_node_type_id,effective_spark_version,enable_elastic_disk,enable_local_disk_encryption,executors,init_scripts_safe_mode,instance_source,jdbc_port,last_activity_time,last_restarted_time,last_state_loss_time,node_type_id,num_workers,runtime_engine,spark_context_id,spark_env_vars,spark_version,start_time,state,state_message,autoscale,custom_tags,policy_id,single_user_name,spark_conf,terminated_time,termination_reason
20,"{'first_on_demand': 1, 'availability': 'SPOT_WITH_FALLBACK', 'zone_id': 'auto', 'spot_bid_price_percent': 10, 'ebs_volume_count': 0}",12.0,0914-130826-ktjg3ao1,93696,debanjoy mukherjee's Cluster,UI,debanjoy.d.mukherjee@koantek.com,USER_ISOLATION,"{'Vendor': 'Databricks', 'Creator': 'debanjoy.d.mukherjee@koantek.com', 'ClusterName': ""debanjoy mukherjee's Cluster"", 'ClusterId': '0914-130826-ktjg3ao1'}",{'disk_count': 0},"{'private_ip': '10.96.222.141', 'node_id': '3e9de1ad84824994bcb5ef624a3d5c04', 'instance_id': 'i-02771faf782de90fa', 'start_timestamp': 1694697094888, 'node_aws_attributes': {'is_spot': False}, 'node_attributes': {'is_spot': False}, 'host_private_ip': '10.96.223.19'}",True,{'node_type_id': 'i3.xlarge'},i3.xlarge,13.3.x-scala2.12,False,False,"[{'private_ip': '10.96.198.220', 'node_id': '2719e8ad0344416bb97ce903ef960698', 'instance_id': 'i-066048b7a6053ea2c', 'start_timestamp': 1694697094967, 'node_aws_attributes': {'is_spot': False}, 'node_attributes': {'is_spot': False}, 'host_private_ip': '10.96.207.74'}, {'private_ip': '10.96.196.6', 'node_id': 'd5282354a1444ac1a70b4276702ffb99', 'instance_id': 'i-0d4d30a493d6fde54', 'start_timestamp': 1694697094926, 'node_aws_attributes': {'is_spot': False}, 'node_attributes': {'is_spot': False}, 'host_private_ip': '10.96.204.132'}]",False,{'node_type_id': 'i3.xlarge'},10000,1694697851409,1694697218736,0,i3.xlarge,2,STANDARD,3891487705972551044,{'PYSPARK_PYTHON': '/databricks/python3/bin/python3'},13.3.x-scala2.12,1694696906611,RUNNING,,null,null,null,null,null,null,null
0,"{'first_on_demand': 1, 'availability': 'SPOT_WITH_FALLBACK', 'zone_id': 'us-west-2b', 'spot_bid_price_percent': 100, 'ebs_volume_type': 'GENERAL_PURPOSE_SSD', 'ebs_volume_count': 3, 'ebs_volume_size': 100}",6.0,0513-131652-gz13auvt,24576,job-1026675266688702-run-96645-aggregations_cluster,JOB,jack.mead@foreveroceans.com,null,"{'Vendor': 'Databricks', 'Creator': 'jack.mead@foreveroceans.com', 'ClusterName': 'job-1026675266688702-run-96645-aggregations_cluster', 'ClusterId': '0513-131652-gz13auvt', 'JobId': '1026675266688702', 'RunName': 'reading_aggregations'}","{'disk_type': {'ebs_volume_type': 'GENERAL_PURPOSE_SSD'}, 'disk_count': 3, 'disk_size': 100}","{'private_ip': '10.96.255.156', 'node_id': '95d0ed542a2946109eef43fc29c5a23d', 'instance_id': 'i-0a11e271d6c087301', 'start_timestamp': 1683984054900, 'node_aws_attributes': {'is_spot': False}, 'node_attributes': {'is_spot': False}, 'host_private_ip': '10.96.238.8'}",True,{'node_type_id': 'm4.large'},m4.large,10.4.x-scala2.12,False,False,"[{'private_ip': '10.96.253.17', 'node_id': 'ff39dba1c5274e8892a879c8177a2053', 'instance_id': 'i-09e1405eff436d386', 'start_timestamp': 1694690117875, 'node_aws_attributes': {'is_spot': True}, 'node_attributes': {'is_spot': True}, 'host_private_ip': '10.96.228.113'}, {'private_ip': '10.96.248.34', 'node_id': 'ffd0c6df471a4d65bca02ee053403119', 'instance_id': 'i-0fae9334cf53d58f1', 'start_timestamp': 1694658013031, 'node_aws_attributes': {'is_spot': True}, 'node_attributes': {'is_spot': True}, 'host_private_ip': '10.96.233.97'}]",False,{'node_type_id': 'm4.large'},10000,1683984196936,1683984243836,0,m4.large,null,STANDARD,4765648694224636634,{'PYSPARK_PYTHON': '/databricks/python3/bin/python3'},10.4.x-scala2.12,1683983812664,RUNNING,,"{'min_workers': 2, 'max_workers': 4, 'target_workers': 2

###Cluster Policies

In [0]:
def get_cluster_policies():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/policies/clusters/list", headers=headers)
    # print('Response status : ', response.status_code)
    cluster_policies_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "policies" in json_response.keys():
                cluster_policies_lst  = json_response["policies"]
            else:
                cluster_policies_lst = [{"message": "an error occured while getting cluster policies"}]
        else:
            cluster_policies_lst = [{"message": "cluster policy not found"}]
    else:
        cluster_policies_lst = [{"message": "an error occured while getting cluster policies"}]
    return cluster_policies_lst

In [0]:
try:
    cluster_policies_lst = get_cluster_policies()
    cluster_policies_df = spark.createDataFrame(cluster_policies_lst)
    print("Cluster Policies:")
    display(cluster_policies_df)
    cluster_policies_path = f"{storage_path}/cluster_policies.parquet"
    cluster_policies_df.write.parquet(cluster_policies_path, mode="overwrite")
except Exception as e:
    print(e)

Cluster Policies:


created_at_timestamp,definition,description,is_default,name,policy_family_id,policy_family_version,policy_id
1667869329000,"{""aws_attributes.spot_bid_price_percent"":{""type"":""fixed"",""value"":100,""hidden"":true},""aws_attributes.availability"":{""type"":""fixed"",""value"":""ON_DEMAND"",""hidden"":true},""node_type_id"":{""type"":""allowlist"",""values"":[""i3.xlarge"",""i3.2xlarge"",""i3.4xlarge"",""i3en.xlarge"",""g4dn.xlarge""],""defaultValue"":""i3.xlarge""},""spark_version"":{""type"":""unlimited"",""defaultValue"":""auto:latest-ml""},""runtime_engine"":{""type"":""fixed"",""value"":""STANDARD"",""hidden"":true},""num_workers"":{""type"":""fixed"",""value"":0,""hidden"":true},""data_security_mode"":{""type"":""allowlist"",""values"":[""SINGLE_USER"",""LEGACY_SINGLE_USER"",""LEGACY_SINGLE_USER_STANDARD""],""defaultValue"":""SINGLE_USER"",""hidden"":true},""driver_instance_pool_id"":{""type"":""forbidden"",""hidden"":true},""cluster_type"":{""type"":""fixed"",""value"":""all-purpose""},""aws_attributes.zone_id"":{""type"":""unlimited"",""defaultValue"":""auto"",""hidden"":true},""instance_pool_id"":{""type"":""forbidden"",""hidden"":true},""spark_conf.spark.databricks.cluster.profile"":{""type"":""fixed"",""value"":""singleNode"",""hidden"":true},""autotermination_minutes"":{""type"":""unlimited"",""defaultValue"":4320,""isOptional"":true},""enable_elastic_disk"":{""type"":""fixed"",""value"":true,""hidden"":true}}",Use with small-to-medium data or libraries like pandas and scikit-learn. Spark runs in local mode.,true,Personal Compute,personal-vm,1,E0631F5C0D002787
1667869329000,"{ ""spark_conf.spark.databricks.cluster.profile"": { ""type"": ""forbidden"", ""hidden"": true }, ""spark_version"": { ""type"": ""unlimited"", ""defaultValue"": ""auto:latest-ml"" }, ""enable_elastic_disk"": { ""type"": ""fixed"", ""value"": true, ""hidden"": true }, ""autotermination_minutes"": { ""type"": ""unlimited"", ""defaultValue"": 120, ""isOptional"": true }, ""node_type_id"": { ""type"": ""unlimited"", ""defaultValue"": ""i3.xlarge"", ""isOptional"": true }, ""autoscale.min_workers"": { ""type"": ""unlimited"", ""defaultValue"": 2 }, ""autoscale.max_workers"": { ""type"": ""unlimited"", ""defaultValue"": 10 }, ""num_workers"": { ""type"": ""forbidden"", ""hidden"": true }, ""aws_attributes.availability"": { ""type"": ""fixed"", ""value"": ""SPOT_WITH_FALLBACK"", ""hidden"": true }, ""aws_attributes.first_on_demand"": { ""type"": ""range"", ""minValue"": 1, ""defaultValue"": 100001 }, ""aws_attributes.zone_id"": { ""type"": ""unlimited"", ""defaultValue"": ""auto"", ""hidden"": true }, ""aws_attributes.spot_bid_price_percent"": { ""type"": ""fixed"", ""value"": 100, ""hidden"": true }, ""instance_pool_id"": { ""type"": ""forbidden"", ""hidden"": true }, ""driver_instance_pool_id"": { ""type"": ""forbidden"", ""hidden"": true }, ""cluster_type"": { ""type"": ""fixed"", ""value"": ""all-purpose"" } }","Run advanced, complex data science projects with dedicated resources.",true,Power User Compute,power-user,1,E0631F5C0D002788
1686253320000,"{ ""spark_conf.spark.databricks.cluster.profile"": { ""type"": ""forbidden"", ""hidden"": true }, ""spark_version"": { ""type"": ""unlimited"", ""defaultValue"": ""auto:latest-lts"" }, ""enable_elastic_disk"": { ""type"": ""fixed"", ""value"": true, ""hidden"": true }, ""autotermination_minutes"": { ""type"": ""unlimited"", ""defaultValue"": 0, ""isOptional"": true }, ""node_type_id"": { ""type"": ""unlimited"", ""defaultValue"": ""i3.2xlarge"", ""isOptional"": true }, ""instance_pool_id"": { ""type"": ""forbidden"", ""hidden"": true }, ""driver_instance_pool_id"": { ""type"": ""forbidden"", ""hidden"": true }, ""autoscale.min_workers"": { ""type"": ""unlimited"", ""defaultValue"": 2 }, ""autoscale.max_workers"": { ""type"": ""unlimited"", ""defaultValue"": 10 }, ""num_workers"": { ""type"": ""forbidden"", ""hidden"": true }, ""aws_attributes.availability"": { ""type"": ""fixed"", ""value"": ""SPOT_

###Cluster Permissions

In [0]:
def get_cluster_permissions(cluster_id):
    response = requests.get(
        f"https://{databricks_host}/api/2.0/permissions/clusters/{cluster_id}", headers=headers)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            cluster_permissions = json_response
        else:
            cluster_permissions = {"message": "cluster permissions not found"}
    else:
        cluster_permissions = {"message": f"an error occured while getting cluster permissions for {cluster_id}"}
    return cluster_permissions

In [0]:
try:
    cluster_permissions_lst = []
    if list(clusters_lst[0].keys())[0] != 'message':
        for cluster in clusters_lst:
            cluster_id = cluster["cluster_id"]
            cluster_permissions_dct = get_cluster_permissions(cluster_id)
            cluster_permissions_dct["cluster_id"] = cluster_id
            cluster_permissions_lst.append(cluster_permissions_dct)
        cluster_permissions_df = spark.createDataFrame(cluster_permissions_lst)
        print("Cluster permissions:")
        # display(cluster_permissions_df)

        cluster_permissions_df = (
        cluster_permissions_df.withColumn("access_control", explode_outer("access_control_list"))
        .select("cluster_id", "access_control")
        # .drop("access_control_list", "object_type", "object_id")
        )
        # print("Relevant Cluster permissions:")
        display(cluster_permissions_df)
        cluster_permissions_path = f"{storage_path}/cluster_permissions.parquet"
        cluster_permissions_df.write.parquet(cluster_permissions_path, mode="overwrite")
    else:
        print("Cluster not found, can't retrieve cluster permissions")
except Exception as e:
    print(e)

Cluster permissions:


cluster_id,access_control
0914-130826-ktjg3ao1,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited=false}], display_name -> debanjoy mukherjee, user_name -> debanjoy.d.mukherjee@koantek.com)"
0914-130826-ktjg3ao1,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/clusters/], inherited=true}], group_name -> admins)"
0513-131652-gz13auvt,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited=false}, {permission_level=CAN_MANAGE, inherited_from_object=[/jobs/1026675266688702], inherited=true}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
0513-131652-gz13auvt,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/clusters/], inherited=true}], group_name -> admins)"
0811-125600-2u5fewpo,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited=false}], display_name -> Mayank Gupta, user_name -> mayank.gupta@foreveroceans.com)"
0811-125600-2u5fewpo,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/clusters/], inherited=true}], group_name -> admins)"
0323-011441-dmm3tjtl,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
0323-011441-dmm3tjtl,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/clusters/], inherited=true}], group_name -> admins)"
0109-200754-jxnrkoic,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
0109-200754-jxnrkoic,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/clusters/], inherited=true}], group_name -> admins)"


###SQL Warehouses

In [0]:
def get_warehouses():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/sql/warehouses", headers=headers)
    # print('Response status : ', response.status_code)
    warehouse_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "warehouses" in json_response.keys():
                warehouse_lst  = json_response["warehouses"]
            else:
                warehouse_lst = [{"message": "an error occured while getting warehouses"}]
        else:
            warehouse_lst = [{"message": "SQL warehouse not found"}]
    else:
        warehouse_lst = [{"message": "an error occured while getting warehouses"}]
    return warehouse_lst

In [0]:
try:
    warehouses_lst = get_warehouses()
    for warehouse in warehouses_lst:
        for key, val in warehouse.items():
            warehouse[key] = str(val)
    warehouses_df = spark.createDataFrame(warehouses_lst)
    print("SQL Warehouses:")
    display(warehouses_df)
    warehouses_path = f"{storage_path}/warehouses.parquet"
    warehouses_df.write.parquet(warehouses_path, mode="overwrite")
except Exception as e:
    print(e)

SQL Warehouses:


auto_resume,auto_stop_mins,channel,cluster_size,creator_id,creator_name,enable_photon,enable_serverless_compute,id,jdbc_url,max_num_clusters,min_num_clusters,name,num_active_sessions,num_clusters,odbc_params,size,spot_instance_policy,state,tags,warehouse_type,health
True,45,{'name': 'CHANNEL_NAME_CURRENT'},2X-Small,1546300086725245,bill.crimmel@foreveroceans.com,True,False,52d9a0099580847c,jdbc:spark://dbc-3c3340bd-60a3.cloud.databricks.com:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/52d9a0099580847c;,1,1,Ashish - PowerBI consulant - OffShore Team,0,0,"{'hostname': 'dbc-3c3340bd-60a3.cloud.databricks.com', 'path': '/sql/1.0/warehouses/52d9a0099580847c', 'protocol': 'https', 'port': 443}",XXSMALL,COST_OPTIMIZED,STOPPED,{},PRO,null
True,45,{'name': 'CHANNEL_NAME_CURRENT'},2X-Small,2008517578761640,john.scharber@foreveroceans.com,True,False,f8bbf72428b6914f,jdbc:spark://dbc-3c3340bd-60a3.cloud.databricks.com:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/f8bbf72428b6914f;,1,1,dev,0,0,"{'hostname': 'dbc-3c3340bd-60a3.cloud.databricks.com', 'path': '/sql/1.0/warehouses/f8bbf72428b6914f', 'protocol': 'https', 'port': 443}",XXSMALL,COST_OPTIMIZED,STOPPED,{},CLASSIC,null
True,45,{'name': 'CHANNEL_NAME_PREVIEW'},2X-Small,5635831880819407,jack.mead@foreveroceans.com,True,False,6491ea9782ee6832,jdbc:spark://dbc-3c3340bd-60a3.cloud.databricks.com:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/6491ea9782ee6832;,1,1,Fivetran,0,0,"{'hostname': 'dbc-3c3340bd-60a3.cloud.databricks.com', 'path': '/sql/1.0/warehouses/6491ea9782ee6832', 'protocol': 'https', 'port': 443}",XXSMALL,COST_OPTIMIZED,STOPPED,{},PRO,null
True,10,null,X-Small,5635831880819407,jack.mead@foreveroceans.com,True,False,54e3962a331ba129,jdbc:spark://dbc-3c3340bd-60a3.cloud.databricks.com:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/54e3962a331ba129;,1,1,FIVETRAN_WAREHOUSE,0,0,"{'hostname': 'dbc-3c3340bd-60a3.cloud.databricks.com', 'path': '/sql/1.0/warehouses/54e3962a331ba129', 'protocol': 'https', 'port': 443}",XSMALL,COST_OPTIMIZED,STOPPED,{},CLASSIC,null
True,45,{'name': 'CHANNEL_NAME_CURRENT'},X-Large,8217428355536273,mathew.goldsborough@foreveroceans.com,True,False,863ee3d38dce246e,jdbc:spark://dbc-3c3340bd-60a3.cloud.databricks.com:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/863ee3d38dce246e;,1,1,PowerBI Warehouse,0,0,"{'hostname': 'dbc-3c3340bd-60a3.cloud.databricks.com', 'path': '/sql/1.0/warehouses/863ee3d38dce246e', 'protocol': 'https', 'port': 443}",XLARGE,COST_OPTIMIZED,STOPPED,{},PRO,null
True,60,null,Small,1268559393822515,nikhilkumar.velluri@foreveroceans.com,True,False,9b5a6f2c1fe03aeb,jdbc:spark://dbc-3c3340bd-60a3.cloud.databricks.com:443/default;transportMode=http;ssl=1;AuthMech=3;httpPath=/sql/1.0/warehouses/9b5a6f2c1fe03aeb;,1,1,Starter Warehouse,0,1,"{'hostname': 'dbc-3c3340bd-60a3.cloud.databricks.com', 'path': '/sql/1.0/warehouses/9b5a6f2c1fe03aeb', 'protocol': 'https', 'port': 443}",SMALL,COST_OPTIMIZED,RUNNING,{},CLASSIC,{'status': 'HEALTHY'}


###SQL Warehouse Permissions

In [0]:
def get_warehouse_permissions(warehouse_id):
    response = requests.get(
        f"https://{databricks_host}/api/2.0/permissions/warehouses/{warehouse_id}", headers=headers)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            warehouse_permissions = json_response
        else:
            warehouse_permissions = {"message": "warehouse permission not found"}
    else:
        warehouse_permissions = {"message": f"an error occured while getting warehouse permissions for {warehouse_id}"}
    return warehouse_permissions

In [0]:
try:
    warehouse_permissions_lst = []
    if list(warehouses_lst[0].keys())[0] != 'message':
        for warehouse in warehouses_lst:
            warehouse_id = warehouse["id"]
            warehouse_permissions_dct = get_warehouse_permissions(warehouse_id)
            warehouse_permissions_dct["warehouse_id"] = warehouse_id
            warehouse_permissions_lst.append(warehouse_permissions_dct)
        warehouse_permissions_df = spark.createDataFrame(warehouse_permissions_lst)
        print("SQL Warehouse permissions:")
        # display(warehouse_permissions_df)

        warehouse_permissions_df = (
        warehouse_permissions_df.withColumn("access_control", explode_outer("access_control_list"))
        .select("warehouse_id", "access_control")
        # .drop("access_control_list", "object_id", "object_type")
        )
        # print("Relevant SQL Warehouse permissions:")
        display(warehouse_permissions_df)
        warehouse_permissions_path = f"{storage_path}/warehouse_permissions.parquet"
        warehouse_permissions_df.write.parquet(warehouse_permissions_path, mode="overwrite")
    else:
        print("SQL Warehouse not found, can't retrieve warehouse permissions")
except Exception as e:
    print(e)

SQL Warehouse permissions:


warehouse_id,access_control
52d9a0099580847c,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Bill Crimmel, user_name -> bill.crimmel@foreveroceans.com)"
52d9a0099580847c,"Map(all_permissions -> [{permission_level=CAN_USE, inherited=false}], display_name -> Mathew Goldsborough, user_name -> mathew.goldsborough@foreveroceans.com)"
52d9a0099580847c,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/sql/warehouses/], inherited=true}], group_name -> admins)"
f8bbf72428b6914f,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> John Scharber, user_name -> john.scharber@foreveroceans.com)"
f8bbf72428b6914f,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/sql/warehouses/], inherited=true}], group_name -> admins)"
f8bbf72428b6914f,"Map(all_permissions -> [{permission_level=CAN_USE, inherited=false}], group_name -> Software-Team)"
6491ea9782ee6832,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
6491ea9782ee6832,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/sql/warehouses/], inherited=true}], group_name -> admins)"
54e3962a331ba129,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
54e3962a331ba129,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited=false}], display_name -> FIVETRAN_USER, service_principal_name -> fc8d01a1-d467-44d0-9678-4fb11cb7dddf)"


###Jobs

In [0]:
def get_jobs():
    response = requests.get(
        f"https://{databricks_host}/api/2.1/jobs/list", headers=headers)
    # print('Response status : ', response.status_code)
    job_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "jobs" in json_response.keys():
                job_lst  = json_response["jobs"]
            else:
                job_lst = [{"message": "an error occured while getting jobs"}]
        else:
            job_lst = [{"message": "job not found"}]
    else:
        job_lst = [{"message": "an error occured while getting jobs"}]
    return job_lst

In [0]:
try:
    jobs_lst = get_jobs()
    jobs_df = spark.createDataFrame(jobs_lst)
    print("Jobs:")
    display(jobs_df)
except Exception as e:
    print(e)

Jobs:


created_time,creator_user_name,job_id,settings
1694632192080,mayank.gupta@foreveroceans.com,605222268861807,"Map(format -> MULTI_TASK, name -> biomass_daily_histograms_PB, email_notifications -> {on_failure=[mayank.gupta@foreveroceans.com], no_alert_for_skipped_runs=true}, timeout_seconds -> 0, max_concurrent_runs -> 1, schedule -> {pause_status=UNPAUSED, timezone_id=America/New_York, quartz_cron_expression=4 0 4 * * ?})"
1694631533827,mayank.gupta@foreveroceans.com,589244123790179,"Map(format -> MULTI_TASK, name -> biomass_daily_weight_histograms_view, email_notifications -> {on_failure=[mayank.gupta@foreveroceans.com], no_alert_for_skipped_runs=true}, timeout_seconds -> 0, max_concurrent_runs -> 1, schedule -> {pause_status=UNPAUSED, timezone_id=America/New_York, quartz_cron_expression=21 0 4 * * ?})"
1694182971934,mayank.gupta@foreveroceans.com,532996036689267,"Map(format -> MULTI_TASK, name -> viewCreationFisheryInsightsJob, email_notifications -> {no_alert_for_skipped_runs=false}, timeout_seconds -> 0, max_concurrent_runs -> 1)"
1692147547553,jack.mead@foreveroceans.com,893527787722201,"Map(format -> MULTI_TASK, name -> Innovasea Ingest, email_notifications -> {no_alert_for_skipped_runs=false}, notification_settings -> {no_alert_for_canceled_runs=false, no_alert_for_skipped_runs=false}, timeout_seconds -> 0, max_concurrent_runs -> 1, schedule -> {pause_status=UNPAUSED, timezone_id=America/Bogota, quartz_cron_expression=8 0 18 * * ?}, webhook_notifications -> {on_failure=[{id=585b4d57-4138-45b4-ac20-a970d047d39b}]})"
1687289056054,jack.mead@foreveroceans.com,256722323464309,"Map(health -> {rules=[{op=GREATER_THAN, metric=RUN_DURATION_SECONDS, value=7200}]}, format -> MULTI_TASK, name -> C2 Readings, email_notifications -> {no_alert_for_skipped_runs=false}, notification_settings -> {no_alert_for_canceled_runs=false, no_alert_for_skipped_runs=false}, duration_warning_seconds -> 7200, timeout_seconds -> 0, max_concurrent_runs -> 1, schedule -> {pause_status=UNPAUSED, timezone_id=America/Bogota, quartz_cron_expression=28 0 18 * * ?}, webhook_notifications -> {on_failure=[{id=585b4d57-4138-45b4-ac20-a970d047d39b}], on_duration_warning_threshold_exceeded=[{id=585b4d57-4138-45b4-ac20-a970d047d39b}]})"
1680231739794,jack.mead@foreveroceans.com,380597932176947,"Map(health -> {rules=[{op=GREATER_THAN, metric=RUN_DURATION_SECONDS, value=2700}]}, format -> MULTI_TASK, name -> Biomass DLT, email_notifications -> {no_alert_for_skipped_runs=false}, notification_settings -> {no_alert_for_canceled_runs=false, no_alert_for_skipped_runs=false}, duration_warning_seconds -> 2700, timeout_seconds -> 0, max_concurrent_runs -> 1, schedule -> {pause_status=UNPAUSED, timezone_id=Pacific/Honolulu, quartz_cron_expression=58 0 14 * * ?}, webhook_notifications -> {on_failure=[{id=585b4d57-4138-45b4-ac20-a970d047d39b}], on_duration_warning_threshold_exceeded=[{id=585b4d57-4138-45b4-ac20-a970d047d39b}]})"
1659124248946,jack.mead@foreveroceans.com,1026675266688702,"Map(format -> MULTI_TASK, name -> reading_aggregations, email_notifications -> {on_failure=[jack.mead@foreveroceans.com], no_alert_for_skipped_runs=false, on_start=[jack.mead@foreveroceans.com], on_success=[jack.mead@foreveroceans.com]}, timeout_seconds -> 0, max_concurrent_runs -> 1)"


###More Details on a Job (like source etc)

In [0]:
def get_job_details(job_id):
    data = {"job_id": job_id}
    response = requests.get(
        f"https://{databricks_host}/api/2.1/jobs/get", headers=headers, json=data)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            job_details = json_response
        else:
            job_details = {"message": "job details not found"}
    else:
        job_details = {"message": f"an error occured while getting jobs details for {job_id}"}
    return job_details

In [0]:
try:
    job_details_lst = []
    if list(jobs_lst[0].keys())[0] != 'message':
        for job in jobs_lst:
            job_id = job["job_id"]
            job_details_dct = get_job_details(job_id)
            job_details_lst.append(job_details_dct)
        job_details_df = spark.createDataFrame(job_details_lst)
        print("Job Details:")
        display(job_details_df)
        job_details_path = f"{storage_path}/jobs.parquet"
        job_details_df.write.parquet(job_details_path, mode="overwrite")
    else:
        print("Job not found, can't get more details")
except Exception as e:
    print(e)

Job Details:


created_time,creator_user_name,job_id,run_as_owner,run_as_user_name,settings
1694632192080,mayank.gupta@foreveroceans.com,605222268861807,true,mayank.gupta@foreveroceans.com,"Map(format -> MULTI_TASK, name -> biomass_daily_histograms_PB, email_notifications -> {on_failure=[mayank.gupta@foreveroceans.com], no_alert_for_skipped_runs=true}, timeout_seconds -> 0, tasks -> [{notebook_task={notebook_path=/Users/mayank.gupta@foreveroceans.com/biomass_daily_histograms_PB, source=WORKSPACE}, task_key=biomass_daily_histograms_PB, email_notifications={}, timeout_seconds=0, existing_cluster_id=0811-125600-2u5fewpo, run_if=ALL_SUCCESS}], max_concurrent_runs -> 1, schedule -> {pause_status=UNPAUSED, timezone_id=America/New_York, quartz_cron_expression=4 0 4 * * ?}, webhook_notifications -> {})"
1694631533827,mayank.gupta@foreveroceans.com,589244123790179,true,mayank.gupta@foreveroceans.com,"Map(format -> MULTI_TASK, name -> biomass_daily_weight_histograms_view, email_notifications -> {on_failure=[mayank.gupta@foreveroceans.com], no_alert_for_skipped_runs=true}, timeout_seconds -> 0, tasks -> [{notebook_task={notebook_path=/Users/mayank.gupta@foreveroceans.com/biomass_daily_weight_histograms_view, source=WORKSPACE}, task_key=biomass_daily_weight_histograms_view, email_notifications={}, timeout_seconds=0, existing_cluster_id=0811-125600-2u5fewpo, run_if=ALL_SUCCESS}], max_concurrent_runs -> 1, schedule -> {pause_status=UNPAUSED, timezone_id=America/New_York, quartz_cron_expression=21 0 4 * * ?}, webhook_notifications -> {})"
1694182971934,mayank.gupta@foreveroceans.com,532996036689267,true,mayank.gupta@foreveroceans.com,"Map(format -> MULTI_TASK, name -> viewCreationFisheryInsightsJob, email_notifications -> {no_alert_for_skipped_runs=false}, timeout_seconds -> 0, tasks -> [{notification_settings={no_alert_for_canceled_runs=false, no_alert_for_skipped_runs=false, alert_on_last_attempt=false}, notebook_task={base_parameters={Reception=7B, broodstockorigin=Ecuador}, source=WORKSPACE, notebook_path=/Users/mayank.gupta@foreveroceans.com/FisheryInsights}, task_key=viewCreationFisheryInsightsJob, email_notifications={}, timeout_seconds=0, existing_cluster_id=0811-125600-2u5fewpo, run_if=ALL_SUCCESS}], max_concurrent_runs -> 1, webhook_notifications -> {})"
1692147547553,jack.mead@foreveroceans.com,893527787722201,true,jack.mead@foreveroceans.com,"Map(format -> MULTI_TASK, name -> Innovasea Ingest, job_clusters -> [{new_cluster={spark_conf={spark.master=local[*, 4], spark.databricks.cluster.profile=singleNode}, cluster_name=, enable_elastic_disk=true, spark_env_vars={PYSPARK_PYTHON=/databricks/python3/bin/python3}, node_type_id=m4.large, data_security_mode=SINGLE_USER, custom_tags={ResourceClass=SingleNode}, num_workers=0, aws_attributes={ebs_volume_count=1, ebs_volume_size=100, zone_id=us-west-2b, spot_bid_price_percent=100, ebs_volume_type=GENERAL_PURPOSE_SSD, availability=SPOT_WITH_FALLBACK, first_on_demand=1}, runtime_engine=STANDARD, spark_version=12.2.x-scala2.12, driver_node_type_id=m4.large}, job_cluster_key=innovasea_source_cluster}], email_notifications -> {no_alert_for_skipped_runs=false}, notification_settings -> {no_alert_for_canceled_runs=false, no_alert_for_skipped_runs=false}, timeout_seconds -> 0, tasks -> [{max_retries=-1, notification_settings={no_alert_for_canceled_runs=false, no_alert_for_skipped_runs=false, alert_on_last_attempt=false}, min_retry_interval_millis=900000, notebook_task={base_parameters={spark.bronze.rollback=604800, spark.bronze.table_path=/delta/innovasea_bronze}, source=GIT, notebook_path=bronze}, task_key=innovasea_bronze, email_notifications={}, health={rules=[{op=GREATER_THAN, metric=RUN_DURATION_SECONDS, value=1800}]}, retry_on_timeout=false, job_cluster_key=innovasea_source_cluster, timeout_seconds=3600, run_if=ALL_SUCCESS}, {depends_on=[{task_key=innovasea_bronze}], max_retries=-1, min_retry_interval_millis=900000, task_key=innovasea_dlt, email_notifications={}, health={rules=[{op=GREATER_THAN, metric=RUN

###Job Permissions

In [0]:
def get_job_permissions(job_id):
    response = requests.get(
        f"https://{databricks_host}/api/2.0/permissions/jobs/{job_id}", headers=headers)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            job_permissions = json_response
        else:
            job_permissions = {"message": "job permission not found"}
    else:
        job_permissions = {"message": f"an error occured while getting job permissions for {job_id}"}
    return job_permissions

In [0]:
try:
    job_permissions_lst = []
    if list(jobs_lst[0].keys())[0] != 'message':
        for job in jobs_lst:
            job_id = job["job_id"]
            job_permissions_dct = get_job_permissions(job_id)
            job_permissions_dct["job_id"] = job_id
            job_permissions_lst.append(job_permissions_dct)
        job_permissions_df = spark.createDataFrame(job_permissions_lst)
        print("Job Permissions:")
        # display(job_permissions_df)

        job_permissions_df = (
        job_permissions_df.withColumn("access_control", explode_outer("access_control_list"))
        .select("job_id", "access_control")
        # .drop("access_control_list", "object_id", "object_type")
        )
        # print("Relevant Job Permissions:")
        display(job_permissions_df)
        job_permissions_path = f"{storage_path}/job_permissions.parquet"
        job_permissions_df.write.parquet(job_permissions_path, mode="overwrite")
    else:
        print("Job not found, can't retrieve job permissions")
except Exception as e:
    print(e)

Job Permissions:


job_id,access_control
605222268861807,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Mayank Gupta, user_name -> mayank.gupta@foreveroceans.com)"
605222268861807,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/jobs/], inherited=true}], group_name -> admins)"
589244123790179,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Mayank Gupta, user_name -> mayank.gupta@foreveroceans.com)"
589244123790179,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/jobs/], inherited=true}], group_name -> admins)"
532996036689267,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Mayank Gupta, user_name -> mayank.gupta@foreveroceans.com)"
532996036689267,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/jobs/], inherited=true}], group_name -> admins)"
893527787722201,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
893527787722201,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/jobs/], inherited=true}], group_name -> admins)"
256722323464309,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
256722323464309,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/jobs/], inherited=true}], group_name -> admins)"


###DLT Pipelines

In [0]:
def get_dlt_pipelines(dlt_pipeline_lst=[], page_token=""):
    if page_token == "":
        response = requests.get(
            f"https://{databricks_host}/api/2.0/pipelines", headers=headers
        )
    else:
        data = {"page_token": page_token}
        response = requests.get(
        f"https://{databricks_host}/api/2.0/pipelines", headers=headers, json=data
        )
    # print('Response status : ', response.status_code)
    # dlt_pipeline_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "statuses" in json_response.keys():
                dlt_pipeline_lst.extend(json_response["statuses"])
                if "next_page_token" in json_response.keys():
                    page_token = json_response["next_page_token"]
                    dlt_pipeline_lst = get_dlt_pipelines(dlt_pipeline_lst, page_token)
            else:
                dlt_pipeline_lst = [{"message": "an error occured while getting dlt pipelines"}]
        else:
            dlt_pipeline_lst = [{"message": "dlt pipeline not found"}]
    else:
        dlt_pipeline_lst = [{"message": "an error occured while getting dlt pipelines"}]
    return dlt_pipeline_lst

In [0]:
try:
    dlt_pipelines_lst = get_dlt_pipelines()
    dlt_pipelines_df = spark.createDataFrame(dlt_pipelines_lst)
    print("DLT Pipelines:")
    display(dlt_pipelines_df)
    dlt_pipelines_path = f"{storage_path}/dlt_pipelines.parquet"
    dlt_pipelines_df.write.parquet(dlt_pipelines_path, mode="overwrite")
except Exception as e:
    print(e)

DLT Pipelines:


creator_user_name,latest_updates,name,pipeline_id,run_as_user_name,state
jack.mead@foreveroceans.com,"List(Map(creation_time -> 2023-09-13T23:00:29.406Z, update_id -> bafd5b33-229a-4133-8b8f-9eccf36dc9fb, state -> COMPLETED), Map(creation_time -> 2023-09-12T23:00:28.849Z, update_id -> 668bed35-78be-4f5c-b9c7-c64768df06fe, state -> COMPLETED), Map(creation_time -> 2023-09-11T23:00:29.634Z, update_id -> 9d506c16-f8e6-4896-aef3-45c94ded08ff, state -> COMPLETED), Map(creation_time -> 2023-09-10T23:00:29.120Z, update_id -> d4c33d70-3ec2-4438-be4c-3e080754dd63, state -> COMPLETED), Map(creation_time -> 2023-09-09T23:00:29.145Z, update_id -> 70331f03-2ebd-4901-9520-1b95a5ab448e, state -> COMPLETED))",C2 Readings,40e35c28-f66e-47b0-9d22-f57c42ce5d7d,jack.mead@foreveroceans.com,IDLE
jack.mead@foreveroceans.com,"List(Map(creation_time -> 2023-09-14T00:00:59.187Z, update_id -> 8a5aee72-ac08-48d5-b569-d9dd4e1c615a, state -> COMPLETED), Map(creation_time -> 2023-09-13T00:01:00.071Z, update_id -> d73ebf99-8824-478c-ab07-77a5871a3851, state -> COMPLETED), Map(creation_time -> 2023-09-12T00:00:59.117Z, update_id -> 17edd37e-ec9b-4d9f-81f7-5ad5134eefb6, state -> COMPLETED), Map(creation_time -> 2023-09-11T00:01:00.187Z, update_id -> 39625d32-308b-4122-965c-b982cb5ecbe1, state -> COMPLETED), Map(creation_time -> 2023-09-10T00:00:59.679Z, update_id -> cd2244b1-e9b2-4dc7-a00a-424b82deaea7, state -> COMPLETED))",Biomass,8c415509-9516-46c0-bb8f-85c4ad285d6b,jack.mead@foreveroceans.com,IDLE
jack.mead@foreveroceans.com,"List(Map(creation_time -> 2023-09-13T23:09:16.286Z, update_id -> 7c272f1b-24aa-42f8-89ed-9b1eed3a1ebe, state -> COMPLETED), Map(creation_time -> 2023-09-12T23:09:58.843Z, update_id -> 0e00ac28-90b5-4a3a-b06d-92fe5db73c05, state -> COMPLETED), Map(creation_time -> 2023-09-11T23:09:23.794Z, update_id -> 479cfb42-f0d3-4a2f-b863-6c4f274bfae4, state -> COMPLETED), Map(creation_time -> 2023-09-10T23:09:26.788Z, update_id -> 23dfbc9e-4dc8-426e-8166-8cecd58baf90, state -> COMPLETED), Map(creation_time -> 2023-09-09T23:09:09.594Z, update_id -> a578ae76-997c-4a79-89e4-d4ffb3e5cd48, state -> COMPLETED))",Innovasea,93b6d190-2017-432a-819c-6a7aaaf71f9c,jack.mead@foreveroceans.com,IDLE
jack.mead@foreveroceans.com,null,Global OpenTelemetry,e2326b27-d5d9-40f1-9b62-6585357a502f,jack.mead@foreveroceans.com,IDLE


###DLT Pipeline Permissions

In [0]:
def get_dlt_pipeline_permissions(pipeline_id):
    response = requests.get(
        f"https://{databricks_host}/api/2.0/permissions/pipelines/{pipeline_id}", headers=headers)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            dlt_pipeline_permissions = json_response
        else:
            dlt_pipeline_permissions = {"message": "dlt permission not found"}
    else:
        dlt_pipeline_permissions = {"message": f"an error occured while getting dlt permissions for {pipeline_id}"}
    return dlt_pipeline_permissions

In [0]:
try:
    dlt_pipeline_permissions_lst = []
    if list(dlt_pipelines_lst[0].keys())[0] != 'message':
        for dlt_pipeline in dlt_pipelines_lst:
            pipeline_id = dlt_pipeline["pipeline_id"]
            dlt_pipeline_permissions_dct = get_dlt_pipeline_permissions(pipeline_id)
            dlt_pipeline_permissions_dct["pipeline_id"] = pipeline_id
            dlt_pipeline_permissions_lst.append(dlt_pipeline_permissions_dct)
        dlt_pipeline_permissions_df = spark.createDataFrame(dlt_pipeline_permissions_lst)
        print("DLT Pipeline Permissions:")
        # display(dlt_pipeline_permissions_df)

        dlt_pipeline_permissions_df = (
        dlt_pipeline_permissions_df.withColumn("access_control", explode_outer("access_control_list"))
        .select("pipeline_id", "access_control")
        # .drop("access_control_list", "object_id", "object_type")
        )
        # print("Relevant DLT Pipeline Permissions:")
        display(dlt_pipeline_permissions_df)
        dlt_pipeline_permissions_path = f"{storage_path}/dlt_pipeline_permissions.parquet"
        dlt_pipeline_permissions_df.write.parquet(dlt_pipeline_permissions_path, mode="overwrite")
    else:
        print("DLT Pipeline not found, can't retrieve dlt permissions")
except Exception as e:
    print(e)

DLT Pipeline Permissions:


pipeline_id,access_control
40e35c28-f66e-47b0-9d22-f57c42ce5d7d,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
40e35c28-f66e-47b0-9d22-f57c42ce5d7d,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/pipelines/], inherited=true}], group_name -> admins)"
8c415509-9516-46c0-bb8f-85c4ad285d6b,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
8c415509-9516-46c0-bb8f-85c4ad285d6b,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/pipelines/], inherited=true}], group_name -> admins)"
93b6d190-2017-432a-819c-6a7aaaf71f9c,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
93b6d190-2017-432a-819c-6a7aaaf71f9c,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/pipelines/], inherited=true}], group_name -> admins)"
e2326b27-d5d9-40f1-9b62-6585357a502f,"Map(all_permissions -> [{permission_level=IS_OWNER, inherited=false}], display_name -> Jack Mead, user_name -> jack.mead@foreveroceans.com)"
e2326b27-d5d9-40f1-9b62-6585357a502f,"Map(all_permissions -> [{permission_level=CAN_MANAGE, inherited_from_object=[/pipelines/], inherited=true}], group_name -> admins)"


###Catalogs

In [0]:
try:
    catalogs_df = spark.sql(f"SHOW CATALOGS")
    display(catalogs_df)
    catalogs_path = f"{storage_path}/catalogs.parquet"
    catalogs_df.write.parquet(catalogs_path, mode="overwrite")
except Exception as e:
    print(e)

catalog
hive_metastore
main
samples
system


###Databases in hive metastore

In [0]:
try:
    spark.sql(f"USE CATALOG hive_metastore")
    databases_df = spark.sql('SHOW DATABASES')
    databases_lst = [db.databaseName for db in databases_df.collect()]
    display(databases_df)
    databases_path = f"{storage_path}/databases.parquet"
    databases_df.write.parquet(databases_path, mode="overwrite")
except Exception as e:
    print(e)

databaseName
_fivetran_setup_test
_fivetran_staging
aquamanager_growout_dbo
aquamanager_hatchery_dbo
brown_bag
default
fivetran_log
limble


###Tables

In [0]:
from pyspark.sql import Row
db_tables_dct = {}
for database in databases_lst:
    spark.sql(f"USE DATABASE {database}")
    tables = [table["tableName"] for table in spark.sql("SHOW TABLES").collect()]
    db_tables_dct[database] = tables
# db_tables_dct

In [0]:
try:
    tables_schema = StructType([
        StructField("Database", StringType()),
        StructField("Tables", ArrayType(StringType()))
      ])
    tables_df = spark.createDataFrame(db_tables_dct.items(), schema=tables_schema)
    tables_df = (
        tables_df.withColumn("Table", explode("Tables"))
        .drop("Tables")
    )
    # display(tables_df)
    # tables_path = f"{storage_path}/tables.parquet"
    # tables_df.write.parquet(tables_path, mode="overwrite")

    detailed_tbl_lst = []
    for table in tables_df.collect():
        db_name = table["Database"]
        tbl_name = table["Table"]
        location, provider, owner, tbl_type = "", "", "", ""
        try:
            ext_tbl_df = spark.sql(f"DESCRIBE TABLE EXTENDED hive_metastore.{db_name}.{tbl_name}")
            for tbl_row in ext_tbl_df.collect():
                if tbl_row["col_name"]=='Location':
                    location = tbl_row["data_type"]
                if tbl_row["col_name"]=='Provider':
                    provider = tbl_row["data_type"]
                if tbl_row["col_name"]=='Owner':
                    owner = tbl_row["data_type"]
                if tbl_row["col_name"]=='Type':
                    tbl_type = tbl_row["data_type"]
        except Exception as e:
            pass
            # print("An error occured while getting table details")

        detailed_tbl_lst.append({"Database": db_name, "Table": tbl_name, "Location": location, "Provider": provider, "Owner": owner, "Table_Type": tbl_type})

    # print(detailed_tbl_lst)
    detailed_tbl_df = spark.createDataFrame(detailed_tbl_lst)
    display(detailed_tbl_df)
    tables_path = f"{storage_path}/tables.parquet"
    tables_df.write.parquet(tables_path, mode="overwrite")
except Exception as e:
    print(e)

Database,Location,Owner,Provider,Table,Table_Type
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/accounttype,root,delta,accounttype,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/actiontrigger,root,delta,actiontrigger,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/activities,root,delta,activities,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/activitytask,root,delta,activitytask,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/activitytaskdetail,root,delta,activitytaskdetail,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/activitytemplatedetail,root,delta,activitytemplatedetail,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/activitytemplatemaster,root,delta,activitytemplatemaster,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/adjustmentfactor,root,delta,adjustmentfactor,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/adjustmenttemplate,root,delta,adjustmenttemplate,MANAGED
aquamanager_growout_dbo,dbfs:/user/hive/warehouse/aquamanager_growout_dbo.db/afimportfromfeeder,root,delta,afimportfromfeeder,MANAGED


###Table Grants

In [0]:
def get_table_grants(db_tables_dct):
    grants_lst = []
    for db, table_lst in db_tables_dct.items():
        for table in table_lst:
            grants_df = spark.sql(f"SHOW GRANTS ON TABLE hive_metastore.{db}.{table}")
            grants_dct = {}
            for grants in grants_df.collect():
                if grants["Principal"] in grants_dct.keys():
                    grants_dct[grants["Principal"]].append({"ActionType":grants["ActionType"], "ObjectType":grants["ObjectType"]})
                else:
                    grants_dct[grants["Principal"]] = [{"ActionType":grants["ActionType"], "ObjectType":grants["ObjectType"]}]

            grants_lst.append({"Database": db, "Table": table, "Grants": [grants_dct]})
    return grants_lst

In [0]:
try:
    if len(db_tables_dct) > 0:
        table_grants_lst = get_table_grants(db_tables_dct)
        # print(table_grants_lst)
        table_grants_df = spark.createDataFrame(table_grants_lst)
        table_grants_df = table_grants_df.select("*", explode_outer("Grants")).drop("Grants")
        table_grants_df = (
            table_grants_df.select("*", explode_outer("col")).drop("col")
            .withColumnRenamed("key", "Principal")
            .withColumnRenamed("value", "Grants")
        )
        display(table_grants_df)
        table_grants_path = f"{storage_path}/table_grants.parquet"
        table_grants_df.write.parquet(table_grants_path, mode="overwrite")
    else:
        print("No tables found, can't retrieve permissions")
except Exception as e:
    print(e)

Database,Table,Principal,Grants
aquamanager_growout_dbo,accounttype,fc8d01a1-d467-44d0-9678-4fb11cb7dddf,"List(Map(ObjectType -> CATALOG$, ActionType -> CREATE), Map(ObjectType -> CATALOG$, ActionType -> USAGE))"
aquamanager_growout_dbo,accounttype,dev.fivetran@foreveroceans.com,"List(Map(ObjectType -> TABLE, ActionType -> OWN))"
aquamanager_growout_dbo,accounttype,read_only,"List(Map(ObjectType -> DATABASE, ActionType -> READ_METADATA), Map(ObjectType -> DATABASE, ActionType -> SELECT), Map(ObjectType -> DATABASE, ActionType -> USAGE), Map(ObjectType -> CATALOG$, ActionType -> SELECT))"
aquamanager_growout_dbo,actiontrigger,fc8d01a1-d467-44d0-9678-4fb11cb7dddf,"List(Map(ObjectType -> CATALOG$, ActionType -> CREATE), Map(ObjectType -> CATALOG$, ActionType -> USAGE))"
aquamanager_growout_dbo,actiontrigger,dev.fivetran@foreveroceans.com,"List(Map(ObjectType -> TABLE, ActionType -> OWN))"
aquamanager_growout_dbo,actiontrigger,read_only,"List(Map(ObjectType -> DATABASE, ActionType -> READ_METADATA), Map(ObjectType -> DATABASE, ActionType -> SELECT), Map(ObjectType -> DATABASE, ActionType -> USAGE), Map(ObjectType -> CATALOG$, ActionType -> SELECT))"
aquamanager_growout_dbo,activities,fc8d01a1-d467-44d0-9678-4fb11cb7dddf,"List(Map(ObjectType -> CATALOG$, ActionType -> CREATE), Map(ObjectType -> CATALOG$, ActionType -> USAGE))"
aquamanager_growout_dbo,activities,dev.fivetran@foreveroceans.com,"List(Map(ObjectType -> TABLE, ActionType -> OWN))"
aquamanager_growout_dbo,activities,read_only,"List(Map(ObjectType -> DATABASE, ActionType -> READ_METADATA), Map(ObjectType -> DATABASE, ActionType -> SELECT), Map(ObjectType -> DATABASE, ActionType -> USAGE), Map(ObjectType -> CATALOG$, ActionType -> SELECT))"
aquamanager_growout_dbo,activitytask,fc8d01a1-d467-44d0-9678-4fb11cb7dddf,"List(Map(ObjectType -> CATALOG$, ActionType -> CREATE), Map(ObjectType -> CATALOG$, ActionType -> USAGE))"


###Mount Points

In [0]:
# try:
#     mount_points = dbutils.fs.mounts()
#     # print(mount_points)
#     mount_points_df = spark.createDataFrame(mount_points)
#     display(mount_points_df)
#     mount_points_path = f"{storage_path}/mount_points.parquet"
#     mount_points_df.write.parquet(mount_points_path, mode="overwrite")
# except Exception as e:
#     print(e)

###SQL Queries

In [0]:
def get_sql_queries():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/preview/sql/queries", headers=headers)
    # print('Response status : ', response.status_code)
    sql_queries_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if json_response["count"] > 0:
            if "results" in json_response.keys():
                sql_queries_lst  = json_response["results"]
            else:
                sql_queries_lst = [{"message": "an error occured while getting sql queries"}]
        else:
            sql_queries_lst = [{"message": "sql query not found"}]
    else:
        sql_queries_lst = [{"message": "an error occured while getting sql queries"}]
    return sql_queries_lst

In [0]:
try:
    sql_queries_lst = get_sql_queries()
    for query in sql_queries_lst:
        for key, val in query.items():
            query[key] = str(val)
    sql_queries_df = spark.createDataFrame(sql_queries_lst)
    print("SQL Queries:")
    display(sql_queries_df)
    sql_queries_path = f"{storage_path}/sql_queries.parquet"
    sql_queries_df.write.parquet(sql_queries_path, mode="overwrite")
except Exception as e:
    print(e)

SQL Queries:


created_at,data_source_id,description,id,is_draft,is_favorite,is_safe,name,options,query,retrieved_at,run_as_role,run_as_service_principal_id,runtime,schedule,tags,updated_at,user,user_id,version
2023-09-12T00:34:00Z,da89e9d7-ea46-49f5-ad03-f14718a098b7,None,be1209be-6e84-4dba-817b-231ebe420b57,False,False,True,Test Query,"{'parent': 'folders/2004961562636373', 'apply_auto_limit': False, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/2615470240180251', 'visualization_control_order': ['6ea471fb-5c5b-4e8a-ae43-3d30031716c2', 'def56daf-6a4e-41ec-b2f8-891cafb9d583', 'e00df68c-f627-40b0-996a-63fe6f978069'], 'catalog': 'hive_metastore', 'schema': 'default', 'parameters': []}","SELECT * FROM water_quality WHERE sensor LIKE 'ysi%' --sensor=""tchain"" AND site_id=""01""",None,None,None,None,None,[],2023-09-12T01:42:47Z,"{'id': 5125916705598599, 'name': 'Brittany Rogers', 'email': 'brittany.rogers@foreveroceans.com'}",5125916705598599,1
2023-09-06T21:56:00Z,da89e9d7-ea46-49f5-ad03-f14718a098b7,None,19e43eba-c391-4dd9-8609-d86037ec8532,False,False,True,sample_query,"{'parent': 'folders/2528178156303120', 'apply_auto_limit': False, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/2044601540104814', 'visualization_control_order': ['a5985bc3-5ba8-4f91-8242-3a70cc5a9a98', 'e10f7f5d-9c20-46ce-8d99-1f748c742f28', 'date_range'], 'parameters': [{'title': 'date_range', 'name': 'date_range', 'type': 'date-range', 'value': 'd_last_90_days'}], 'schema': 'default'}","SELECT site_id, sensor, reading, window.start as `timestamp`, metrics.avg as value FROM -- hive_metastore.aquamanager_growout_dbo.view_cohort_details water_quality_aggregations WHERE window.start BETWEEN '{{ date_range.start }}' AND '{{ date_range.end }}' AND metrics.avg > 7 ORDER BY year",None,None,None,None,None,[],2023-09-12T22:28:39Z,"{'id': 5635831880819407, 'name': 'Jack Mead', 'email': 'jack.mead@foreveroceans.com'}",5635831880819407,1
2023-09-01T19:39:23Z,da89e9d7-ea46-49f5-ad03-f14718a098b7,None,28d637f7-1d96-4d0a-aea2-a59eb244490e,False,False,True,view_stocking_import_transactions,"{'parent': 'folders/194734588111596', 'apply_auto_limit': False, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/2236068880653708', 'visualization_control_order': [], 'parameters': [], 'schema': 'aquamanager_growout_dbo'}","CREATE OR REPLACE VIEW view_stocking_import_transactions AS SELECT date_format(cltrans.transdate, ""d/M/yyyy"") as date, cage.designation as Unit, lot.designation as Batch, cltransdetails.fishintransaction as `Fish No`, NULL as `Av. Weight`, 'Seriola rivoliana' as Species, NULL as SFR, NULL as FCR, hatchery.designation as Hatchery, NULL as `Fry Cost / Fish`, NULL as Comments FROM cltrans, cltransdetails, cage, cagelot, lot, transkind, hatchery left join cause on cltransdetails.causeid = cause.causeid WHERE cltrans.cltransid = cltransdetails.cltransid and hatchery.hatcheryid = cltrans.hatcheryid and cltransdetails.cagelotid = cagelot.cagelotid and cagelot.lotid = lot.lotid and cage.cageid = cagelot.cageid and cltrans.transkindid = transkind.transkindid and cltrans.transkindid in (1, 3, 5, 6, 7, 8, 12) ORDER BY lot.designation, cage.designation, cltrans.transdate, cltrans.transkindid",None,None,None,None,None,[],2023-09-01T22:51:00Z,"{'id': 5635831880819407, 'name': 'Jack Mead', 'email': 'jack.mead@foreveroceans.com'}",5635831880819407,1
2023-09-01T19:07:28Z,da89e9d7-ea46-49f5-ad03-f14718a098b7,None,b465d5a9-c5f4-4d42-bc5a-5556cc7855f8,False,False,True,view_harvesting_import_transactions,"{'parent': 'folders/194734588111596', 'apply_auto_limit': True, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/1882118701116161', 'visualization_control_order': [], 'parameters': [], 'schema': 'aquamanager_growout_dbo'}","CREATE OR REPLACE VIEW view_harvesting_import_transactions AS SELECT concat(cage.designation, ' / ', lot.designation) as `Unit / Batch`, date_format(transdate, 'd/M/yyyy') as Date, 'Seriola rivoliana' as 

###SQL Dashboards

In [0]:
def get_sql_dashboards():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/preview/sql/dashboards", headers=headers)
    # print('Response status : ', response.status_code)
    sql_dashboards_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if json_response["count"] > 0:
            if "results" in json_response.keys():
                sql_dashboards_lst  = json_response["results"]
            else:
                sql_dashboards_lst = [{"message": "an error occured while getting sql dashboards"}]
        else:
            sql_dashboards_lst = [{"message": "sql dashboard not found"}]
    else:
        sql_dashboards_lst = [{"message": "an error occured while getting sql dashboards"}]
    return sql_dashboards_lst

In [0]:
try:
    sql_dashboards_lst = get_sql_dashboards()
    for dashboard in sql_dashboards_lst:
        for key, val in dashboard.items():
            dashboard[key] = str(val)
    sql_dashboards_df = spark.createDataFrame(sql_dashboards_lst)
    print("SQL Dashboards:")
    display(sql_dashboards_df)
    sql_dashboards_path = f"{storage_path}/sql_dashboards.parquet"
    sql_dashboards_df.write.parquet(sql_dashboards_path, mode="overwrite")
except Exception as e:
    print(e)

SQL Dashboards:


color_palette,created_at,dashboard_filters_enabled,data_source_id,id,is_draft,is_favorite,name,options,refresh_schedules,run_as_role,run_as_service_principal_id,slug,tags,updated_at,user,user_id,version,warehouse_id,widgets
None,2023-07-18T03:36:52Z,False,da89e9d7-ea46-49f5-ad03-f14718a098b7,915425dd-2732-4d08-8319-35d86763e94b,False,False,Exosonde All Data_Reading Table,"{'parent': 'folders/684333318728758', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/3316363798785463'}",[],None,None,exosonde-all-data_reading-table,[],2023-07-18T03:55:56Z,"{'id': 5880560177636254, 'name': 'Robert Baikie', 'email': 'robert.baikie@foreveroceans.com'}",5880560177636254,3,9b5a6f2c1fe03aeb,None
None,2023-07-16T04:07:38Z,False,da89e9d7-ea46-49f5-ad03-f14718a098b7,38f2e70e-9808-420e-915d-b0a1f27b46b3,False,False,Exosonde All Data_WQ Table,"{'parent': 'folders/684333318728758', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/946082583803848'}",[],None,None,exosonde-all-data_wq-table,[],2023-07-16T04:14:40Z,"{'id': 5880560177636254, 'name': 'Robert Baikie', 'email': 'robert.baikie@foreveroceans.com'}",5880560177636254,3,9b5a6f2c1fe03aeb,None
None,2023-07-14T22:01:04Z,False,da89e9d7-ea46-49f5-ad03-f14718a098b7,9867181f-8682-4061-88bc-5b9487990cfb,False,False,All FO WQ by Site ID,"{'parent': 'folders/684333318728758', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/1362751060864613'}",[],None,None,all-fo-wq-by-site-id,[],2023-07-14T23:20:51Z,"{'id': 5880560177636254, 'name': 'Robert Baikie', 'email': 'robert.baikie@foreveroceans.com'}",5880560177636254,3,9b5a6f2c1fe03aeb,None
"['#077A9D', '#FFAB00', '#00A972', '#FF3621']",2023-07-12T23:17:57Z,False,da89e9d7-ea46-49f5-ad03-f14718a098b7,5a2a718c-9476-4188-81c0-183bda2d54b1,False,False,TChain - Readings Table,"{'parent': 'folders/684333318728758', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/2212198285575324'}",[],None,None,tchain---readings-table,[],2023-07-13T00:27:29Z,"{'id': 5880560177636254, 'name': 'Robert Baikie', 'email': 'robert.baikie@foreveroceans.com'}",5880560177636254,6,9b5a6f2c1fe03aeb,None
None,2023-07-11T19:00:29Z,False,None,be1b5ba5-96a0-4380-b375-0dfc5e4149f9,False,False,Exosonde Temp & Depth,"{'parent': 'folders/684333318728758', 'run_as_role': 'owner', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/2381111862836106'}",[],owner,None,exosonde-temp-depth,[],2023-07-12T00:59:40Z,"{'id': 5880560177636254, 'name': 'Robert Baikie', 'email': 'robert.baikie@foreveroceans.com'}",5880560177636254,4,None,None
None,2023-07-11T07:20:40Z,False,da89e9d7-ea46-49f5-ad03-f14718a098b7,fcf440a3-7f8a-4d3a-9263-54489a8e01ad,False,False,TChain Data_WQ Table,"{'parent': 'folders/684333318728758', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/221502799692040'}",[],None,None,tchain-data_wq-table,[],2023-07-16T03:21:45Z,"{'id': 5880560177636254, 'name': 'Robert Baikie', 'email': 'robert.baikie@foreveroceans.com'}",5880560177636254,4,9b5a6f2c1fe03aeb,None
None,2023-07-11T01:43:15Z,False,da89e9d7-ea46-49f5-ad03-f14718a098b7,dd01fbb0-567b-4916-bb62-95b8ce5164fd,False,False,test,"{'parent': 'folders/2887721657226725', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/699731017729072'}",[],None,None,test,[],2023-07-11T01:43:26Z,"{'id': 4080374272075846, 'name': 'Devin Dennis', 'email': 'devin.dennis@foreveroceans.com'}",4080374272075846,3,9b5a6f2c1fe03aeb,None
None,2023-06-07T22:22:03Z,True,da89e9d7-ea46-49f5-ad03-f14718a098b7,1b02cee8-23a2-4b4b-a5fc-640aff5c0a2c,False,False,Biocam / Sampling Variation,"{'parent': 'folders/727987725580313', 'refresh_schedules': [], 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/2068006880659679'}",[],None,None,biocam-sampling-variation,[],2023-08-16T18:19

###SQL Alerts

In [0]:
def get_sql_alerts():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/preview/sql/alerts", headers=headers)
    # print('Response status : ', response.status_code)
    sql_alerts_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            sql_alerts_lst  = json_response
        else:
            sql_alerts_lst = [{"message": "sql alert not found"}]
    else:
        sql_alerts_lst = [{"message": "an error occured while getting sql alerts"}]
    return sql_alerts_lst

In [0]:
try:
    sql_alerts_lst = get_sql_alerts()
    for alert in sql_alerts_lst:
        for key, val in alert.items():
            alert[key] = str(val)
    sql_alerts_df = spark.createDataFrame(sql_alerts_lst)
    print("SQL Alerts:")
    display(sql_alerts_df)
    sql_alerts_path = f"{storage_path}/sql_alerts.parquet"
    sql_alerts_df.write.parquet(sql_alerts_path, mode="overwrite")
except Exception as e:
    print(e)

SQL Alerts:


conditions,created_at,id,last_triggered_at,name,options,query,rearm,refresh_schedules,state,subscriptions,updated_at,user,user_id,warehouse_id
"{'op': '<', 'alert': {'column': {'name': 'instance_count', 'display': 'instance_count', 'aggregation': None}}, 'threshold': {'value': '1000'}, 'query_plan': None, 'empty_result_state': 'triggered'}",2023-08-11T22:03:58Z,026ae1e7-475a-4c80-a3a1-ec42d9fa9edf,2023-08-15T01:01:11Z,PTY04 Biocam Instance Count Today: instance_count < 1000,"{'muted': False, 'notify_on_ok': True, 'schedule_failures': 0, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/4267468832516120', 'parent': 'folders/727987725580313', 'op': '<', 'column': 'instance_count', 'display_column': 'instance_count', 'aggregation': None, 'value': '1000', 'query_plan': None, 'empty_result_state': 'triggered'}","{'warehouse_id': '9b5a6f2c1fe03aeb', 'id': '0a03ae0d-5886-4c07-bd68-3d16b1464ee3', 'name': 'PTY04 Biocam Instance Count Today', 'description': None, 'query': ""SELECT\n date_format(CAST(timestamp as DATE), 'yyyy-MM-dd') as date,\n element_at(split(unique_key, '/'), 2) as site_id,\n approx_count_distinct(concat(unique_key, '/', instance_index)) as instance_count\nFROM\n biomass_filtered\nWHERE\n element_at(split(unique_key, '/'), 2) = '04'\n AND date_format(CAST(timestamp as DATE), 'yyyy-MM-dd') = curdate() - 1\nGROUP BY\n site_id,\n date\nORDER BY\n date DESC\nLIMIT\n 1"", 'is_draft': False, 'updated_at': '2023-08-16T18:18:03Z', 'created_at': '2023-08-11T22:00:34Z', 'data_source_id': 'da89e9d7-ea46-49f5-ad03-f14718a098b7', 'options': {'parent': 'folders/727987725580313', 'apply_auto_limit': True, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/4267468832516115', 'visualization_control_order': [], 'parameters': []}, 'version': 1, 'tags': [], 'is_safe': True, 'user_id': 5635831880819407, 'run_as_role': None, 'run_as_service_principal_id': None, 'schedule': None, 'is_archived': False}",None,"[{'id': '74e197e5-40d6-4e00-beaf-092991b9366b', 'cron': '0 0 1 */1 * ?', 'job_id': '8b0b49c1a6ec07b2a8ce9b34bd98293e7b2affe0', 'data_source_id': 'da89e9d7-ea46-49f5-ad03-f14718a098b7'}]",triggered,[{'destination_id': '585b4d57-4138-45b4-ac20-a970d047d39b'}],2023-09-14T01:00:57Z,"{'id': 5635831880819407, 'name': 'Jack Mead', 'email': 'jack.mead@foreveroceans.com'}",5635831880819407,9b5a6f2c1fe03aeb
"{'op': '<', 'alert': {'column': {'name': 'total_frames', 'display': 'total_frames', 'aggregation': None}}, 'threshold': {'value': '100000'}, 'query_plan': None, 'empty_result_state': 'triggered'}",2023-08-11T22:29:37Z,1e622c78-3f41-4ee6-867f-2a5e2b164a59,2023-08-15T01:01:47Z,PTY02 Biocam Processed Frames Today: total_frames < 100000,"{'muted': False, 'notify_on_ok': True, 'schedule_failures': 0, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/4267468832516128', 'parent': 'folders/727987725580313', 'op': '<', 'column': 'total_frames', 'display_column': 'total_frames', 'aggregation': None, 'value': '100000', 'query_plan': None, 'empty_result_state': 'triggered'}","{'warehouse_id': '9b5a6f2c1fe03aeb', 'id': 'd4d71ff5-c995-4514-adcc-c819228a2033', 'name': 'PTY02 Biocam Processed Frames Today', 'description': None, 'query': ""WITH data AS (\n SELECT\n date_format(\n CAST(\n from_unixtime(timestamp / 1000, 'yyyy-MM-dd') as DATE\n ),\n 'yyyy-MM-dd'\n ) as date,\n approx_count_distinct(uniqueKey) as total_frames,\n element_at(split(uniqueKey, '/'), 2) as site_id\n FROM\n biomass_results\n GROUP BY\n site_id,\n date\n)\nSELECT\n date,\n total_frames,\n site_id\nFROM\n data\nWHERE\n site_id = '02'\n AND date = curdate() - 1\nORDER BY\n date desc\nLIMIT\n 1"", 'is_draft': False, 'updated_at': '2023-08-16T18:19:23Z', 'created_at': '2023-08-11T22:23:54Z', 'data_source_id': 'da89e9d7-ea46-49f5-ad03-f14718a098b7', 'options': {'parent': 'folders/727987725580313', 'apply_auto_limit': True, 'folder_node_status': 'ACTIVE', 'folder_node_internal_name': 'tree/4267468832516123', 'visualization_control_order': [],

###Permissions related to SQL Queries, Dashboards, Alerts

In [0]:
def get_sql_permissions(object_type, object_id):
    response = requests.get(
        f"https://{databricks_host}/api/2.0/preview/sql/permissions/{object_type}/{object_id}", headers=headers)
    # print('Response status : ', response.status_code)
    sql_permissions_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            sql_permissions_lst  = json_response
        else:
            sql_permissions_lst = {"message": "sql permission not found"}
    else:
        sql_permissions_lst = {"message": "an error occured while getting sql permissions"}
    return sql_permissions_lst

In [0]:
try:
    sql_permissions_lst = []
    if list(sql_queries_lst[0].keys())[0] != 'message':
        for query in sql_queries_lst:
            query_id = query["id"]
            query_dct = get_sql_permissions("queries", query_id)
            query_dct["id"] = query_id
            sql_permissions_lst.append(query_dct)
    else:
        print("SQL query not found, can't retrieve permissions")
    
    if list(sql_dashboards_lst[0].keys())[0] != 'message':
        for dashboard in sql_dashboards_lst:
            dashboard_id = dashboard["id"]
            dashboard_dct = get_sql_permissions("dashboards", dashboard_id)
            dashboard_dct["id"] = dashboard_id
            sql_permissions_lst.append(dashboard_dct)
    else:
        print("SQL Dashboards not found, can't retrieve permissions")
    
    if list(sql_alerts_lst[0].keys())[0] != 'message':
        for alert in sql_alerts_lst:
            alert_id = alert["id"]
            alert_dct = get_sql_permissions("alerts", alert_id)
            alert_dct["id"] = alert_id
            sql_permissions_lst.append(alert_dct)
    else:
        print("SQL Alerts not found, can't retrieve permissions")

    if len(sql_permissions_lst) > 0:
        sql_permissions_df = spark.createDataFrame(sql_permissions_lst)
        sql_permissions_df = sql_permissions_df.drop("object_id")
        print("SQL Permissions:")
        display(sql_permissions_df)
        sql_permissions_path = f"{storage_path}/sql_permissions.parquet"
        sql_permissions_df.write.parquet(sql_permissions_path, mode="overwrite")
    else:
        print("SQL Permission not found")
except Exception as e:
    print(e)

SQL Permissions:


access_control_list,id,object_type
"List(Map(permission_level -> CAN_MANAGE, user_name -> brittany.rogers@foreveroceans.com), Map(permission_level -> CAN_MANAGE, group_name -> admins))",be1209be-6e84-4dba-817b-231ebe420b57,query
"List(Map(permission_level -> CAN_MANAGE, user_name -> brittany.rogers@foreveroceans.com), Map(permission_level -> CAN_MANAGE, user_name -> robert.baikie@foreveroceans.com), Map(permission_level -> CAN_MANAGE, user_name -> jack.mead@foreveroceans.com), Map(permission_level -> CAN_MANAGE, group_name -> admins))",19e43eba-c391-4dd9-8609-d86037ec8532,query
"List(Map(permission_level -> CAN_MANAGE, group_name -> users), Map(permission_level -> CAN_MANAGE, group_name -> admins))",28d637f7-1d96-4d0a-aea2-a59eb244490e,query
"List(Map(permission_level -> CAN_MANAGE, group_name -> users), Map(permission_level -> CAN_MANAGE, group_name -> admins))",b465d5a9-c5f4-4d42-bc5a-5556cc7855f8,query
"List(Map(permission_level -> CAN_MANAGE, group_name -> users), Map(permission_level -> CAN_MANAGE, group_name -> admins))",d31593da-5e1b-4e29-bb29-9e2180a379b6,query
"List(Map(permission_level -> CAN_MANAGE, group_name -> users), Map(permission_level -> CAN_MANAGE, group_name -> admins))",eae3b9d4-54de-4032-aa48-52fba562ff3b,query
"List(Map(permission_level -> CAN_MANAGE, group_name -> users), Map(permission_level -> CAN_MANAGE, group_name -> admins))",374cc62c-72e2-4910-be70-90b751028c4a,query
"List(Map(permission_level -> CAN_MANAGE, user_name -> jack.mead@foreveroceans.com), Map(permission_level -> CAN_MANAGE, group_name -> admins))",9c9ccbae-3101-48cb-a291-7e64068e6e98,query
"List(Map(permission_level -> CAN_MANAGE, user_name -> jack.mead@foreveroceans.com), Map(permission_level -> CAN_MANAGE, group_name -> admins))",0ad3574d-2b60-40cd-9677-cd4bf0e2075b,query
"List(Map(permission_level -> CAN_MANAGE, user_name -> jack.mead@foreveroceans.com), Map(permission_level -> CAN_MANAGE, group_name -> admins))",d4d71ff5-c995-4514-adcc-c819228a2033,query


###Instance Profile

In [0]:
def get_instance_profiles():
    response = requests.get(
        f"https://{databricks_host}/api/2.0/instance-profiles/list", headers=headers)
    # print('Response status : ', response.status_code)
    instance_profiles_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "instance_profiles" in json_response.keys():
                instance_profiles_lst  = json_response["instance_profiles"]
            else:
                instance_profiles_lst = [{"message": "an error occured while getting instance-profiles"}]
        else:
            instance_profiles_lst = [{"message": "instance-profile not found"}]
    else:
        instance_profiles_lst = [{"message": "an error occured while getting instance-profiles"}]
    return instance_profiles_lst

In [0]:
try:
    instance_profiles_lst = get_instance_profiles()
    instance_profiles_df = spark.createDataFrame(instance_profiles_lst)
    print("Instance-Profiles:")
    display(instance_profiles_df)
    instance_profiles_path = f"{storage_path}/instance_profiles.parquet"
    instance_profiles_df.write.parquet(instance_profiles_path, mode="overwrite")
except Exception as e:
    print(e)

Instance-Profiles:


message
instance-profile not found


###Experiments

In [0]:
def get_experiments(experiments_lst=[], page_token=""):
    if page_token == "":
        response = requests.get(
            f"https://{databricks_host}/api/2.0/mlflow/experiments/list", headers=headers
        )
    else:
        data = {"page_token": page_token}
        response = requests.get(
        f"https://{databricks_host}/api/2.0/mlflow/experiments/list", headers=headers, json=data
        )
    # print('Response status : ', response.status_code)
    # experiments_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "experiments" in json_response.keys():
                experiments_lst.extend(json_response["experiments"])
                if "next_page_token" in json_response.keys():
                    page_token = json_response["next_page_token"]
                    experiments_lst = get_experiments(experiments_lst, page_token)
            else:
                experiments_lst = [{"message": "an error occured while getting experiments"}]
        else:
            experiments_lst = [{"message": "experiment not found"}]
    else:
        experiments_lst = [{"message": "an error occured while getting experiments"}]
    return experiments_lst

In [0]:
try:
    experiments_lst = get_experiments()
    experiments_df = spark.createDataFrame(experiments_lst)
    print("Experiments:")
    display(experiments_df)
    experiments_path = f"{storage_path}/experiments.parquet"
    experiments_df.write.parquet(experiments_path, mode="overwrite")
except Exception as e:
    print(e)

Experiments:


message
experiment not found


###Experiment Permissions

In [0]:
def get_experiment_permissions(experiment_id):
    response = requests.get(
        f"https://{databricks_host}/api/2.0/permissions/experiments/{experiment_id}", headers=headers)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            experiment_permissions = json_response
        else:
            experiment_permissions = {"message": "experiment permissions not found"}
    else:
        experiment_permissions = {"message": f"an error occured while getting experiment permissions for {experiment_id}"}
    return experiment_permissions

In [0]:
try:
    experiment_permissions_lst = []
    if list(experiments_lst[0].keys())[0] != 'message':
        for experiment in experiments_lst:
            experiment_id = experiment["experiment_id"]
            experiment_permissions_dct = get_experiment_permissions(experiment_id)
            experiment_permissions_dct["experiment_id"] = experiment_id
            experiment_permissions_lst.append(experiment_permissions_dct)
        experiment_permissions_df = spark.createDataFrame(experiment_permissions_lst)
        experiment_permissions_df = experiment_permissions_df.select("experiment_id", "access_control_list")
        print("Experiment Permissions:")
        display(experiment_permissions_df)
    else:
        print("Experiment not found, can't retrieve experiment permissions")
except Exception as e:
    print(e)

Experiment not found, can't retrieve experiment permissions


###Registered Models

In [0]:
def get_registered_models(registered_models_lst=[], page_token=""):
    if page_token == "":
        response = requests.get(
            f"https://{databricks_host}/api/2.0/mlflow/registered-models/list", headers=headers
        )
    else:
        data = {"page_token": page_token}
        response = requests.get(
        f"https://{databricks_host}/api/2.0/mlflow/registered-models/list", headers=headers, json=data
        )
    # print('Response status : ', response.status_code)
    # registered_models_lst = []
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "registered_models" in json_response.keys():
                registered_models_lst.extend(json_response["registered_models"])
                if "next_page_token" in json_response.keys():
                    page_token = json_response["next_page_token"]
                    registered_models_lst = get_registered_models(registered_models_lst, page_token)
            else:
                registered_models_lst = [{"message": "an error occured while getting registered models"}]
        else:
            registered_models_lst = [{"message": "no registered model found"}]
    else:
        registered_models_lst = [{"message": "an error occured while getting registered models"}]
    return registered_models_lst

In [0]:
def get_registered_model_details(model_name):
    data = {"name" : model_name}
    response = requests.get(
        f"https://{databricks_host}/api/2.0/mlflow/databricks/registered-models/get", headers=headers, json=data)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            if "registered_model_databricks" in json_response:
                model_details = json_response["registered_model_databricks"]
            else:
                model_details = {"message": f"an error occured while getting models details for {model_name}"}
        else:
            model_details = {"message": "model details not found"}
    else:
        model_details = {"message": f"an error occured while getting models details for {model_name}"}
    return model_details

In [0]:
try:
    registered_model_details_lst = []
    registered_models_lst = get_registered_models()
    # registered_models_df = spark.createDataFrame(registered_models_lst)
    if list(registered_models_lst[0].keys())[0] != "message":
        for registered_model in registered_models_lst:
            model_name = registered_model["name"]
            registered_model_details_dct = get_registered_model_details(model_name)
            registered_model_details_lst.append(registered_model_details_dct)
        registered_models_df = spark.createDataFrame(registered_model_details_lst)
        print("Registered Models:")
        display(registered_models_df)
        registered_models_path = f"{storage_path}/registered_models.parquet"
        registered_models_df.write.parquet(registered_models_path, mode="overwrite")
    else:
        print("Registered model not found, can't retrieve more details")
except Exception as e:
    print(e)

Registered model not found, can't retrieve more details


###Registered Model Permissions

In [0]:
def get_registered_model_permissions(registered_model_id):
    response = requests.get(
        f"https://{databricks_host}/api/2.0/permissions/registered-models/{registered_model_id}", headers=headers)
    # print('Response status : ', response.status_code)
    if response.status_code == 200:
        json_response = response.json()
        if len(json_response) > 0:
            registered_model_permissions = json_response
        else:
            registered_model_permissions = {"message": "registered model permissions not found"}
    else:
        registered_model_permissions = {"message": f"an error occured while getting registered model permissions for {registered_model_id}"}
    return registered_model_permissions

In [0]:
try:
    registered_model_permissions_lst = []
    if len(registered_model_details_lst) > 0:
        for registered_model in registered_model_details_lst:
            registered_model_id = registered_model["id"]
            registered_model_permissions_dct = get_registered_model_permissions(registered_model_id)
            registered_model_permissions_dct["registered_model_id"] = registered_model_id
            registered_model_permissions_lst.append(registered_model_permissions_dct)
        registered_model_permissions_df = spark.createDataFrame(registered_model_permissions_lst)
        registered_model_permissions_df = registered_model_permissions_df.select("registered_model_id","access_control_list")
        print("Registered Model Permissions:")
        display(registered_model_permissions_df)
    else:
        print("Registered models not found, can't retrieve model permissions")
except Exception as e:
    print(e)

Registered models not found, can't retrieve model permissions
